In [19]:
import settrade_v2

In [39]:
from settrade_v2 import Investor

investor = Investor(
                app_id="QxHlBT7kNFlZt8M6",                                 
                app_secret="GRrzAlrGUgCzR70gU1VPeTYg7fSzU/mmmk2omvyYBLA=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="Ameen-D")            

account_info = deri.get_account_info()

account_info                                              

{'creditLine': 2000000000.0,
 'excessEquity': 2000000000.0,
 'cashBalance': 2000000000.0,
 'equity': 2000000000.0,
 'totalMR': 0.0,
 'totalMM': 0.0,
 'totalFM': 0.0,
 'callForceFlag': 'No',
 'callForceMargin': 0.0,
 'liquidationValue': 2000000000.0,
 'depositWithdrawal': 0.0,
 'callForceMarginMM': 0.0,
 'initialMargin': 0.0,
 'closingMethod': 'Auto Net'}

In [ ]:
import psycopg2

# ข้อมูลการเชื่อมต่อ
host = "localhost"           # หรือ IP เช่น "127.0.0.1"
port = "54322"
database = "candlestick"
user = "postgres"            # ✅ แก้จาก posgres เป็น postgres
password = "asdzxc123"

conn = None
cursor = None

try:
    # เชื่อมต่อ
    conn = psycopg2.connect(
        host=host,
        port=port,
        database=database,
        user=user,
        password=password
    )
    cursor = conn.cursor()

    # สร้างตาราง
    create_table_query = """
    CREATE TABLE IF NOT EXISTS candlestick_data (
        symbol TEXT,
        time TIMESTAMP,
        open_price FLOAT,
        high_price FLOAT,
        low_price FLOAT,
        close_price FLOAT,
        volume BIGINT,
        value FLOAT,
        PRIMARY KEY (symbol, time)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()

    print("✅ ตาราง 'candlestick_data' สร้างสำเร็จ!")

except Exception as e:
    print("❌ เกิดข้อผิดพลาด:", e)

finally:
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()


✅ ตาราง 'candlestick_data' สร้างสำเร็จ!


In [ ]:
import psycopg2


In [53]:
#Get Candlestick
market = investor.MarketData()
res = market.get_candlestick(
symbol="AOT",
interval="1m",
normalized=True,)


In [ ]:
import pandas as pd

In [54]:
if isinstance(res, dict) and "data" in res:
    raw_data = res["data"]
else:
    raw_data = res  # กรณี res เป็น list เลย
df = pd.DataFrame(raw_data)
df

,lastSequence,time,open,high,low,close,volume,value
0,3132682,1759812540,39.00,39.25,39.00,39.00,8100,0.0
1,3132682,1759812600,39.25,39.25,39.25,39.25,3800,0.0
2,3132682,1759812660,39.00,39.25,39.00,39.25,900,0.0
3,3132682,1759812720,39.00,39.00,39.00,39.00,9100,0.0
4,3132682,1759812780,39.25,39.25,39.25,39.25,300,0.0
...,...,...,...,...,...,...,...,...
495,3132682,1759915560,39.75,39.75,39.50,39.75,22200,882425.0
496,3132682,1759915620,39.75,39.75,39.75,39.75,205700,8176575.0
497,3132682,1759915680,39.75,39.75,39.75,39.75,26800,1065300.0
498,3132682,1759915740,39.50,39.75,39.50,39.75,54300,2155875.0


In [ ]:
import os
import time
import psycopg2
import pandas as pd
from settrade_v2 import Investor
from dotenv import load_dotenv
from datetime import datetime

# ✅ โหลดตัวแปรจาก .env
load_dotenv()

# ✅ ดึงค่าจาก .env
db_config = {
    "host": "localhost",
    "port": "54322",  # ✅ ตรงกับที่คุณตั้งไว้
    "database": "candlestick",
    "user": "postgres",
    "password": "asdzxc123"
}

# ✅ สร้าง Investor Object สำหรับเรียก Market Data
investor = Investor(
                app_id="QxHlBT7kNFlZt8M6",                                 
                app_secret="GRrzAlrGUgCzR70gU1VPeTYg7fSzU/mmmk2omvyYBLA=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="Ameen-D")    
market = investor.MarketData()

# ✅ ฟังก์ชันดึงข้อมูล Financial Data และบันทึกลง PostgreSQL
def insert_financial_data(symbol):
    try:
        print(f"🔍 กำลังดึงข้อมูล Financial Data สำหรับ {symbol}...")
        
        # ✅ ดึงข้อมูลจาก Settrade API
        res = market.get_quote_symbol(symbol)

        if not isinstance(res, dict) or "symbol" not in res:
            print(f"⚠️ ไม่มีข้อมูลสำหรับ {symbol}.")
            return

        # ✅ เตรียมข้อมูล
        timestamp = datetime.now()
        open_price = res.get("open", 0.0)
        high_price = res.get("high", 0.0)
        low_price = res.get("low", 0.0)
        close_price = res.get("last", 0.0)
        volume = res.get("totalVolume", 0)
        value = res.get("totalValue", 0.0)

        # ✅ เชื่อมต่อฐานข้อมูลและบันทึกข้อมูล
        with psycopg2.connect(**db_config) as conn:
            with conn.cursor() as cursor:
                insert_query = """
                    INSERT INTO candlestick_data (symbol, time, open_price, high_price, low_price, close_price, volume, value)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """
                cursor.execute(insert_query, (
                    symbol, timestamp, open_price, high_price,
                    low_price, close_price, volume,  
                    
                    value
                ))
                conn.commit()
                print(f"✅ ข้อมูลของ {symbol} บันทึกเรียบร้อยแล้ว!")

    except Exception as e:
        print(f"❌ เกิดข้อผิดพลาดกับ {symbol}: {e}")

# ✅ โหลดรายชื่อหุ้นทั้งหมดจากไฟล์ CSV
try:
    symbols_df = pd.read_excel("stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ 'stocth_names.xlsx' ใช้รายชื่อหุ้นที่กำหนดแทน")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# ✅ ดึงข้อมูล Financial Data สำหรับหุ้นทุกตัว
for symbol in symbols:
    insert_financial_data(symbol)
    time.sleep(2)  # หน่วงเวลาเพื่อไม่ให้ API ถูก block


🔍 กำลังดึงข้อมูล Financial Data สำหรับ 24CS...
✅ ข้อมูลของ 24CS บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ 2S...
✅ ข้อมูลของ 2S บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ 3BBIF...
✅ ข้อมูลของ 3BBIF บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ A...
✅ ข้อมูลของ A บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ A5...
✅ ข้อมูลของ A5 บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ AAI...
✅ ข้อมูลของ AAI บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ AAV...
✅ ข้อมูลของ AAV บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ABM...
✅ ข้อมูลของ ABM บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACAP...
✅ ข้อมูลของ ACAP บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACC...
✅ ข้อมูลของ ACC บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACE...
✅ ข้อมูลของ ACE บันทึกเรียบร้อยแล้ว!
🔍 กำลังดึงข้อมูล Financial Data สำหรับ ACG...
✅ ข้อมูลของ ACG บันทึกเรียบร้อยแล้ว!
🔍 กำ

KeyboardInterrupt: 

In [21]:
import psycopg2
import os
from dotenv import load_dotenv

# ✅ โหลด .env (ถ้ามี)
load_dotenv()

# ✅ ตั้งค่าการเชื่อมต่อ PostgreSQL
db_config = {
    "host": os.getenv("DB_HOST", "localhost"),
    "port": os.getenv("DB_PORT", "54322"),
    "database": os.getenv("DB_NAME", "candlestick"),
    "user": os.getenv("DB_USER", "postgres"),
    "password": os.getenv("DB_PASSWORD", "asdzxc123")
}

# ✅ คำสั่ง CREATE TABLE
create_table_query = """
CREATE TABLE IF NOT EXISTS financal_data_fromsettradeapi (
    symbol TEXT PRIMARY KEY,
    aumSize FLOAT,
    average FLOAT,
    change FLOAT,
    eps FLOAT,
    exchange TEXT,
    exercisePrice FLOAT,
    exerciseRatio FLOAT,
    high FLOAT,
    impliedVolatility FLOAT,
    inav FLOAT,
    instrumentType TEXT,
    intrinsicValue FLOAT,
    last FLOAT,
    lastTradingDate DATE,
    low FLOAT,
    marketStatus TEXT,
    maturityDate DATE,
    moneyness TEXT,
    pbv FLOAT,
    pe FLOAT,
    percentChange FLOAT,
    percentYield FLOAT,
    securityType TEXT,
    status TEXT,
    theoretical FLOAT,
    toLastTrade FLOAT,
    totalVolume BIGINT,
    totalBuyVolume BIGINT,
    totalNoSideVolume BIGINT,
    totalSellVolume BIGINT,
    underlying TEXT,
    underlyingPrice FLOAT
);
"""

# ✅ รันคำสั่งสร้างตาราง
try:
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    cursor.execute(create_table_query)
    conn.commit()
    print("✅ ตาราง financal_data_fromsettradeapi ถูกสร้างเรียบร้อยแล้ว (หรือมีอยู่แล้ว)")
except Exception as e:
    print(f"❌ เกิดข้อผิดพลาด: {e}")
finally:
    if cursor: cursor.close()
    if conn: conn.close()


✅ ตาราง financal_data_fromsettradeapi ถูกสร้างเรียบร้อยแล้ว (หรือมีอยู่แล้ว)


In [24]:
import os
import time
import psycopg2
import pandas as pd
from dotenv import load_dotenv
from settrade_v2 import Investor
from datetime import datetime

# ✅ โหลดตัวแปรจาก .env
load_dotenv()

# ✅ ตั้งค่าการเชื่อมต่อ PostgreSQL
db_config = {
    "host": os.getenv("DB_HOST", "localhost"),
    "port": os.getenv("DB_PORT", "54322"),
    "database": os.getenv("DB_NAME", "candlestick"),
    "user": os.getenv("DB_USER", "postgres"),
    "password": os.getenv("DB_PASSWORD", "asdzxc123")
}

investor = Investor(
                app_id="QxHlBT7kNFlZt8M6",                                 
                app_secret="GRrzAlrGUgCzR70gU1VPeTYg7fSzU/mmmk2omvyYBLA=", 
                broker_id="SANDBOX",
                app_code="SANDBOX",
                is_auto_queue = False)

deri = investor.Derivatives(account_no="Ameen-D")

# ✅ ฟังก์ชันป้องกัน error field
def safe_get(data, key, default=None):
    val = data.get(key, default)
    return val if isinstance(val, (int, float, str, type(None))) else default

# ✅ แปลง timestamp เป็นวันที่
def parse_date(timestamp):
    if isinstance(timestamp, (int, float)):
        return datetime.fromtimestamp(timestamp).date()
    return None

# ✅ ฟังก์ชันดึงและบันทึกข้อมูล
def insert_financial_data(symbol):
    try:
        print(f"🔍 ดึงข้อมูล {symbol}...")
        market = investor.MarketData()
        res = market.get_quote_symbol(symbol)

        if not isinstance(res, dict) or "symbol" not in res:
            print(f"⚠️ ไม่มีข้อมูล: {symbol}")
            return

        # ✅ เชื่อมต่อฐานข้อมูล
        with psycopg2.connect(**db_config) as conn:
            with conn.cursor() as cursor:
                cursor.execute("""
                    INSERT INTO financal_data_fromsettradeapi (
                        symbol, aumSize, average, change, eps, exchange,
                        exercisePrice, exerciseRatio, high, impliedVolatility,
                        inav, instrumentType, intrinsicValue, last,
                        lastTradingDate, low, marketStatus, maturityDate,
                        moneyness, pbv, pe, percentChange, percentYield,
                        securityType, status, theoretical, toLastTrade,
                        totalVolume, totalBuyVolume, totalNoSideVolume,
                        totalSellVolume, underlying, underlyingPrice
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                              %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                              %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                              %s, %s, %s)
                    ON CONFLICT (symbol) DO NOTHING;
                """, (
                    symbol,
                    safe_get(res, 'aumSize'),
                    safe_get(res, 'average'),
                    safe_get(res, 'change'),
                    safe_get(res, 'eps'),
                    safe_get(res, 'exchange'),
                    safe_get(res, 'exercisePrice'),
                    safe_get(res, 'exerciseRatio'),
                    safe_get(res, 'high'),
                    safe_get(res, 'impliedVolatility'),
                    safe_get(res, 'inav'),
                    safe_get(res, 'instrumentType'),
                    safe_get(res, 'intrinsicValue'),
                    safe_get(res, 'last'),
                    parse_date(res.get('lastTradingDate')),
                    safe_get(res, 'low'),
                    safe_get(res, 'marketStatus'),
                    parse_date(res.get('maturityDate')),
                    safe_get(res, 'moneyness'),
                    safe_get(res, 'pbv'),
                    safe_get(res, 'pe'),
                    safe_get(res, 'percentChange'),
                    safe_get(res, 'percentYield'),
                    safe_get(res, 'securityType'),
                    safe_get(res, 'status'),
                    safe_get(res, 'theoretical'),
                    safe_get(res, 'toLastTrade'),
                    safe_get(res, 'totalVolume'),
                    safe_get(res, 'totalBuyVolume'),
                    safe_get(res, 'totalNoSideVolume'),
                    safe_get(res, 'totalSellVolume'),
                    safe_get(res, 'underlying'),
                    safe_get(res, 'underlyingPrice')
                ))

        print(f"✅ บันทึกข้อมูล {symbol} สำเร็จ")

    except Exception as e:
        print(f"❌ ERROR [{symbol}]: {e}")

# ✅ โหลดรายชื่อหุ้นจาก Excel
try:
    symbols_df = pd.read_excel("stocth_names.xlsx")
    symbols = symbols_df['หลักทรัพย์'].dropna().tolist()
except FileNotFoundError:
    print("❌ ไม่พบไฟล์ stocth_names.xlsx → ใช้ชุดหุ้นตัวอย่าง")
    symbols = ["PTT", "AOT", "SCB", "CPALL", "ADVANC"]

# 🔁 ดึงและบันทึกข้อมูล
for symbol in symbols:
    insert_financial_data(symbol)
    time.sleep(1.5)

print("\n🎉 เสร็จสิ้นการโหลดและบันทึกข้อมูล Fundamental ทุกตัว!")


🔍 ดึงข้อมูล 24CS...
✅ บันทึกข้อมูล 24CS สำเร็จ
🔍 ดึงข้อมูล 2S...
✅ บันทึกข้อมูล 2S สำเร็จ
🔍 ดึงข้อมูล 3BBIF...
✅ บันทึกข้อมูล 3BBIF สำเร็จ
🔍 ดึงข้อมูล A...
✅ บันทึกข้อมูล A สำเร็จ
🔍 ดึงข้อมูล A5...
✅ บันทึกข้อมูล A5 สำเร็จ
🔍 ดึงข้อมูล AAI...
✅ บันทึกข้อมูล AAI สำเร็จ
🔍 ดึงข้อมูล AAV...
✅ บันทึกข้อมูล AAV สำเร็จ
🔍 ดึงข้อมูล ABM...
✅ บันทึกข้อมูล ABM สำเร็จ
🔍 ดึงข้อมูล ACAP...
✅ บันทึกข้อมูล ACAP สำเร็จ
🔍 ดึงข้อมูล ACC...
✅ บันทึกข้อมูล ACC สำเร็จ
🔍 ดึงข้อมูล ACE...
✅ บันทึกข้อมูล ACE สำเร็จ
🔍 ดึงข้อมูล ACG...
✅ บันทึกข้อมูล ACG สำเร็จ
🔍 ดึงข้อมูล ADB...
✅ บันทึกข้อมูล ADB สำเร็จ
🔍 ดึงข้อมูล ADD...
✅ บันทึกข้อมูล ADD สำเร็จ
🔍 ดึงข้อมูล ADVANC...
✅ บันทึกข้อมูล ADVANC สำเร็จ
🔍 ดึงข้อมูล ADVICE...
✅ บันทึกข้อมูล ADVICE สำเร็จ
🔍 ดึงข้อมูล AE...
✅ บันทึกข้อมูล AE สำเร็จ
🔍 ดึงข้อมูล AEONTS...
✅ บันทึกข้อมูล AEONTS สำเร็จ
🔍 ดึงข้อมูล AF...
✅ บันทึกข้อมูล AF สำเร็จ
🔍 ดึงข้อมูล AFC...
❌ ERROR [AFC]: Symbol not found
🔍 ดึงข้อมูล AGE...
✅ บันทึกข้อมูล AGE สำเร็จ
🔍 ดึงข้อมูล AH...
✅ บันทึกข้อมูล AH

In [25]:
create_table_query_bid_ask_timeseries = """
CREATE TABLE IF NOT EXISTS bid_ask_timeseries (
    time TIMESTAMPTZ NOT NULL,
    symbol TEXT NOT NULL,

    bid_flag TEXT,
    ask_flag TEXT,

    bid_price1 FLOAT, bid_volume1 INT,
    bid_price2 FLOAT, bid_volume2 INT,
    bid_price3 FLOAT, bid_volume3 INT,
    bid_price4 FLOAT, bid_volume4 INT,
    bid_price5 FLOAT, bid_volume5 INT,
    bid_price6 FLOAT, bid_volume6 INT,
    bid_price7 FLOAT, bid_volume7 INT,
    bid_price8 FLOAT, bid_volume8 INT,
    bid_price9 FLOAT, bid_volume9 INT,
    bid_price10 FLOAT, bid_volume10 INT,

    ask_price1 FLOAT, ask_volume1 INT,
    ask_price2 FLOAT, ask_volume2 INT,
    ask_price3 FLOAT, ask_volume3 INT,
    ask_price4 FLOAT, ask_volume4 INT,
    ask_price5 FLOAT, ask_volume5 INT,
    ask_price6 FLOAT, ask_volume6 INT,
    ask_price7 FLOAT, ask_volume7 INT,
    ask_price8 FLOAT, ask_volume8 INT,
    ask_price9 FLOAT, ask_volume9 INT,
    ask_price10 FLOAT, ask_volume10 INT,

    PRIMARY KEY (time, symbol)
);
"""



 

In [49]:
import os
import psycopg2

# เชื่อมต่อฐานข้อมูลจาก environment หรือ fallback เป็น default
conn = psycopg2.connect(
    host=os.getenv("DB_HOST", "localhost"),
    port=os.getenv("DB_PORT", "54322"),
    dbname=os.getenv("DB_NAME", "candlestick"),
    user=os.getenv("DB_USER", "postgres"),
    password=os.getenv("DB_PASSWORD", "asdzxc123")
)

# ✅ ประกาศ cursor ให้ถูกต้อง
cur = conn.cursor()

# ✅ Execute CREATE TABLE
cur.execute(create_table_query_bid_ask_timeseries)

conn.commit()


In [41]:
import time
import psycopg2
from datetime import datetime
from settrade_v2 import Investor

# ---------- เชื่อมต่อ Settrade API ----------

realtime = investor.RealtimeDataConnection()

# ---------- เชื่อมต่อ PostgreSQL ----------

# ---------- คำสั่ง INSERT ----------
insert_query = """
INSERT INTO bid_ask_timeseries (
    time, symbol, bid_flag, ask_flag,
    bid_price1, bid_volume1, bid_price2, bid_volume2, bid_price3, bid_volume3,
    bid_price4, bid_volume4, bid_price5, bid_volume5, bid_price6, bid_volume6,
    bid_price7, bid_volume7, bid_price8, bid_volume8, bid_price9, bid_volume9,
    bid_price10, bid_volume10,
    ask_price1, ask_volume1, ask_price2, ask_volume2, ask_price3, ask_volume3,
    ask_price4, ask_volume4, ask_price5, ask_volume5, ask_price6, ask_volume6,
    ask_price7, ask_volume7, ask_price8, ask_volume8, ask_price9, ask_volume9,
    ask_price10, ask_volume10
)
VALUES (%s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s,
        %s, %s,
        %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s,
        %s, %s)
"""

# ---------- Callback เมื่อมีข้อมูลใหม่ ----------
def my_message(result):
    if not result or "data" not in result:
        return

    data_dict = result["data"]
    symbol = data_dict.get("symbol", "UNKNOWN")

    # เตรียมข้อมูลใส่ใน tuple ตามลำดับ
    insert_data = (
        datetime.now(),
        symbol,
        data_dict.get("bid_flag", None),
        data_dict.get("ask_flag", None)
    )

    # เพิ่ม bid_price1-10 และ bid_volume1-10
    for i in range(1, 11):
        insert_data += (
            data_dict.get(f"bid_price{i}", 0.0),
            data_dict.get(f"bid_volume{i}", 0)
        )

    # เพิ่ม ask_price1-10 และ ask_volume1-10
    for i in range(1, 11):
        insert_data += (
            data_dict.get(f"ask_price{i}", 0.0),
            data_dict.get(f"ask_volume{i}", 0)
        )

    try:
        cur.execute(insert_query, insert_data)
        conn.commit()
        print(f"[✅ INSERTED] {symbol} @ {datetime.now().strftime('%H:%M:%S')}")
    except Exception as e:
        print("[❌ ERROR]", e)
        conn.rollback()

# ---------- เริ่ม subscribe ----------
sub = realtime.subscribe_bid_offer("AOT", on_message=my_message)
sub.start()

# ---------- ให้รันตลอด ----------
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("👋 ปิดการทำงาน realtime...")
    sub.close()
    cur.close()
    conn.close()


[❌ ERROR] current transaction is aborted, commands ignored until end of transaction block

👋 ปิดการทำงาน realtime...


AttributeError: 'Subscriber' object has no attribute 'close'

In [42]:
import pandas as pd

# โหลดไฟล์
df = pd.read_excel("stocth_names.xlsx")

# แสดงข้อมูล
print(df.head())


  หลักทรัพย์
0       24CS
1         2S
2      3BBIF
3          A
4         A5


In [50]:
import psycopg2


# สร้าง cursor
cursor = conn.cursor()

# คำสั่ง SQL สำหรับสร้างตาราง
create_table_query = """
CREATE TABLE IF NOT EXISTS stock_symbols (
    id SERIAL PRIMARY KEY,
    symbol TEXT
);
"""

# รันคำสั่ง SQL
cursor.execute(create_table_query)

# ยืนยันการเปลี่ยนแปลง
conn.commit()



print("สร้างตาราง stock_symbols เรียบร้อยแล้ว")


สร้างตาราง stock_symbols เรียบร้อยแล้ว


In [51]:
# Insert ข้อมูล
for symbol in df['หลักทรัพย์']:
    cursor.execute("INSERT INTO stock_symbols (symbol) VALUES (%s)", (symbol,))

conn.commit()

In [52]:
realtime = investor.RealtimeDataConnection()

def my_message(result):
    print(result)

sub = realtime.subscribe_bid_offer("AOT", on_message = my_message)
sub.start()

# run main thread forever
while True:
    time.sleep(1)

{'is_success': True, 'data': {'symbol': 'AOT', 'bid_flag': 'NORMAL', 'ask_flag': 'NORMAL', 'bid_price1': 39.5, 'bid_price2': 39.25, 'bid_price3': 39.0, 'bid_price4': 38.75, 'bid_price5': 38.5, 'bid_price6': 38.25, 'bid_price7': 38.0, 'bid_price8': 37.75, 'bid_price9': 37.5, 'bid_price10': 37.25, 'ask_price1': 39.75, 'ask_price2': 40.0, 'ask_price3': 40.25, 'ask_price4': 40.5, 'ask_price5': 40.75, 'ask_price6': 41.0, 'ask_price7': 41.25, 'ask_price8': 41.5, 'ask_price9': 41.75, 'ask_price10': 42.0, 'bid_volume1': 405000, 'bid_volume2': 1234600, 'bid_volume3': 1731400, 'bid_volume4': 1298000, 'bid_volume5': 1088500, 'bid_volume6': 831700, 'bid_volume7': 840300, 'bid_volume8': 778600, 'bid_volume9': 268600, 'bid_volume10': 224200, 'ask_volume1': 363900, 'ask_volume2': 676700, 'ask_volume3': 1237100, 'ask_volume4': 947500, 'ask_volume5': 606200, 'ask_volume6': 672500, 'ask_volume7': 268700, 'ask_volume8': 504100, 'ask_volume9': 335700, 'ask_volume10': 408300}}


KeyboardInterrupt: 